## Utilities

In [ ]:
import math
import random

import numpy as np
import pandas as pd
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from sklearn_extra.cluster import KMedoids

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import gurobipy as gp
from gurobipy import GRB

import pickle
from itertools import permutations, combinations, chain
from tqdm import tqdm

from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cityblock
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from seaborn import histplot

from Clustering_Functions import csv_parse, HH_proxy, Borda_vector

## Distance Histograms

In [7]:
_, election, _, _ = csv_parse('scot-elex-main/edinburgh_2017_ward2.csv')

In [ ]:
model_types = ['continuous', 'continuous_rest', 'discrete']
embedding_types = ['hh', 'bordaP', 'bordaA']

all_ballots = list(election.keys())
num_ballots = len(all_ballots)
num_cands = len(set([item for ranking in all_ballots for item in ranking]))

weights = np.array([election[ballot] for ballot in all_ballots])
VALUE_SETS = {'hh': np.array([-1, 0, 1]), 'bordaP': np.array(range(num_cands)), 'bordaA': np.array([l/2 for l in range(1, 2*num_cands + 1)])}
BALLOTS = {'hh': 2*np.array([HH_proxy(ballot,num_cands=num_cands) for ballot in all_ballots]), 
           'bordaP': np.array([Borda_vector(ballot, num_cands = num_cands, borda_style='pes') for ballot in all_ballots]), 
           'bordaA': np.array([Borda_vector(ballot, num_cands = num_cands, borda_style = 'avg') for ballot in all_ballots]) - 1 # Adjust for bordaA
           }

DIMS = {k: range(BALLOTS[k].shape[1]) for k in BALLOTS.keys()}
K = range(2)

# Data directories
RESULTS_DIRECTORY = 'results'
CLUSTER_HISTOGRAM_DIRECTORY = f'{RESULTS_DIRECTORY}/cluster_histograms'
ALL_POINTS_HISTOGRAM_DIRECTORY = f'{RESULTS_DIRECTORY}/all_ballots_histograms'


In [89]:
def get_centroids_discrete(model, ballots):
    def strip_non_numeric(text):
    # Join all characters that are digits
        return int(''.join(char for char in str(text) if char.isdigit()))
    centroids = []
    for var in model.getVars():
        if var.VarName.startswith('isCenter') and var.Start == 1:
            centroids.append(ballots[strip_non_numeric(var.VarName)])
    return np.array(centroids)

def extract_centroids(model, z, K, Dims, V):
    import numpy as np
    
    # Initialize centroid array
    centroids = np.zeros((len(K), len(Dims)))
    
    # For each cluster and dimension
    for r in K:
        for i in Dims:
            # Find the value v where z[i,r,v] = 1
            for v in V:
                if abs(z[i,r,v] - 1.0) < 1e-6:  # Check if binary variable is 1 (with tolerance)
                    centroids[r,i] = v
                    break
    
    return centroids

def extract_z_variables(model, Dims, K, V):
    # Initialize the result dictionary
    result = {}
    
    # Create a mapping of variable names to their expected tuples
    name_to_tuple = {f"z[{i},{r},{v}]": (i, r, v) 
                    for i in Dims for r in K for v in V}
    
    # Iterate through variables and extract binary values
    for var in model.getVars():
        
        if var.VarName in name_to_tuple:
            indices = name_to_tuple[var.VarName]
            # Round to ensure we get exactly 0 or 1
            # Why .Start? Not sure, for some reason .X doesn't work, maybe try running model.optimize() first
            # Nice thing is when you load solutions from file, the variables start with .start
            result[indices] = round(var.Start)
    
    return result

def expand_weighted_ballots(ballot_matrix, weights):
    """
    Expands a weighted ballot matrix into individual ballots.
    
    Parameters:
    -----------
    ballot_matrix : numpy.ndarray
        A matrix where each row represents a unique ballot pattern
        Shape: (n_patterns, n_candidates)
    weights : numpy.ndarray
        An array containing the count/weight of each ballot pattern
        Shape: (n_patterns,)
        
    Returns:
    --------
    numpy.ndarray
        A matrix where each row represents an individual ballot
        Shape: (total_ballots, n_candidates)
    """
    # Convert weights to integers if they aren't already
    weights = np.round(weights).astype(int)
    
    # Calculate the total number of individual ballots
    total_ballots = np.sum(weights)
    
    # Initialize the output matrix
    # Shape will be (total_ballots, n_candidates)
    expanded_ballots = np.zeros((total_ballots, ballot_matrix.shape[1]), 
                              dtype=ballot_matrix.dtype)
    
    # Keep track of where we are in the output matrix
    current_position = 0
    
    # For each ballot pattern and its weight
    for ballot, weight in zip(ballot_matrix, weights):
        # Calculate the end position for this batch of identical ballots
        end_position = current_position + weight
        
        # Fill in the identical ballots
        expanded_ballots[current_position:end_position] = ballot
        
        # Update our position for the next batch
        current_position = end_position
        
    return expanded_ballots

def assign_to_clusters(points, centroids):
    """
    Assigns each point to its nearest centroid.
    
    Parameters:
    points: numpy array of shape (n_samples, n_features)
    centroids: numpy array of shape (n_clusters, n_features)
    
    Returns:
    numpy array of shape (n_samples,) containing cluster assignments
    """
    n_points = len(points)
    labels = np.zeros(n_points, dtype=int)
    
    # For each point, find the closest centroid
    for i, point in enumerate(points):
        # Calculate distances to all centroids
        distances = [cityblock(point, centroid) for centroid in centroids]
        # Assign to closest centroid
        labels[i] = np.argmin(distances)
        
    return labels

def load_model(model_type, embedding_type, num_clusters):
    model = gp.read(f'models/{model_type}_{embedding_type}_{num_clusters}.mps')
    model.read(f'models/{model_type}_{embedding_type}_{num_clusters}.sol')
    model.update()
    return model

def load_get_centroids(model_type, embedding_type, num_clusters):
    K = range(num_clusters)
    model = load_model(model_type, embedding_type, num_clusters)

    if model_type == 'discrete':
        centroids = get_centroids_discrete(model, BALLOTS[embedding_type])
    else:
        z = extract_z_variables(model, DIMS[embedding_type], K, VALUE_SETS[embedding_type])
        centroids = extract_centroids(model, z, K, DIMS[embedding_type], VALUE_SETS[embedding_type])

    return centroids
    

In [90]:
centroids = {}
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            loaded_centroids = load_get_centroids(model_type, embedding_type, num_clusters)
            centroids[(model_type, embedding_type, num_clusters)] = loaded_centroids - 1 if embedding_type == 'bordaA' else loaded_centroids # Adjust for bordaA

Read MPS format model from file models/continuous_hh_2.mps
Reading time = 0.01 seconds
continuous_k_medians: 1911 rows, 2770 columns, 59090 nonzeros
Read solution from file models/continuous_hh_2.sol
Read MPS format model from file models/continuous_bordaP_2.mps
Reading time = 0.00 seconds
continuous_k_medians: 1743 rows, 2686 columns, 25518 nonzeros
Read solution from file models/continuous_bordaP_2.sol
Read MPS format model from file models/continuous_bordaA_2.mps
Reading time = 0.01 seconds
continuous_k_medians: 2233 rows, 2882 columns, 34240 nonzeros
Read solution from file models/continuous_bordaA_2.sol
Read MPS format model from file models/continuous_rest_hh_2.mps
Reading time = 0.01 seconds
continuous_rest_k_medians: 2499 rows, 2868 columns, 60392 nonzeros
Read solution from file models/continuous_rest_hh_2.sol
Read MPS format model from file models/continuous_rest_bordaP_2.mps
Reading time = 0.00 seconds
continuous_k_medians: 1583 rows, 2686 columns, 24370 nonzeros
Read soluti

In [91]:
def cluster_distance_histogram(ballots, centroids, weights, bins, x_ticks, save_location, show=True):
    # Calculate distances to each centroid
    expanded_ballots = expand_weighted_ballots(ballots, weights)
    distances = cdist(expanded_ballots, centroids, metric='cityblock')
    n_centroids = centroids.shape[0]
    
    # Find nearest centroid for each point
    nearest_centroid = np.argmin(distances, axis=1)
    
    # Create subplots
    fig, axes = plt.subplots(n_centroids, 1, figsize=(12, 4*n_centroids))
    if n_centroids == 1:
        axes = [axes]
    
    max_distance = np.max(distances)
    # Create histogram for each centroid
    for i in range(n_centroids):
        ax = axes[i]
        
        # Get distances to current centroid for all points
        all_distances = distances[:, i]
        
        # Create masks for assigned and unassigned points
        assigned_mask = (nearest_centroid == i)

        df = pd.DataFrame({'Distance': all_distances, 'In This Cluster': assigned_mask})

        # Plot histograms with different colors
        # Points assigned to this cluster
        histplot(df, x='Distance', ax=ax, bins=bins[bins <= max_distance + 1], 
                stat='proportion', color='blue', alpha=0.6,
                hue='In This Cluster', common_norm=True)
        
        ax.set_title(f'Centroid {i+1} Distance Distribution')
        ax.set_xticks(x_ticks[x_ticks <= max_distance])
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{save_location}.png')
    if show:
        plt.show()
    plt.close()

In [92]:
bin_edges = {
    'hh': np.arange(-0.5, num_cands*num_cands, 1),
    'bordaP': np.arange(-0.5, num_cands*num_cands, 1),
    'bordaA': np.arange(-0.25, num_cands*num_cands, 0.5)
}
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            cluster_distance_histogram(BALLOTS[embedding_type], centroids[(model_type, embedding_type, num_clusters)], weights, bin_edges[embedding_type], np.arange(0, num_cands*num_cands, step=2), f"{CLUSTER_HISTOGRAM_DIRECTORY}/{model_name}_cluster_distance_hist", show=False)

In [93]:
def all_distance_histogram(ballots, centroids, weights, bins, x_ticks, save_location, show=True):
    expanded_ballots = expand_weighted_ballots(ballots, weights)
    n_centroids = centroids.shape[0]
    
    # Create subplots
    fig, axes = plt.subplots(n_centroids, 1, figsize=(12, 4*n_centroids))
    if n_centroids == 1:
        axes = [axes]
    
    distances = cdist(expanded_ballots, centroids, metric='cityblock')
    df = pd.DataFrame(distances, columns=[f'Centroid {i}' for i in range(1, n_centroids+1)])
    
    # Cut off bins at max distance
    max_distance = np.max(distances)
    bins = bins[bins <= max_distance + 1]
    x_ticks = x_ticks[x_ticks <= max_distance]
    
    for i, ax in enumerate(axes):
        histplot(df.iloc[:, i], bins=bins, stat='proportion', ax=ax)
        ax.set_title(f'All Ballots Distance Distribution to Centroid {i+1}')
        ax.set_xticks(x_ticks)
        ax.set_xlabel('Distance')
        ax.set_ylabel('Proportion')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{save_location}.png')
    if show:
        plt.show()
    plt.close()

In [94]:
bin_edges = {
    'hh': np.arange(-0.5, num_cands*num_cands, 1),
    'bordaP': np.arange(-0.5, num_cands*num_cands, 1),
    'bordaA': np.arange(-0.25, num_cands*num_cands, 0.5)
}

for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            all_distance_histogram(BALLOTS[embedding_type], centroids[(model_type, embedding_type, num_clusters)], weights, bin_edges[embedding_type], np.arange(0, num_cands*num_cands, step=2), f"{ALL_POINTS_HISTOGRAM_DIRECTORY}/{model_name}_all_distance_hist", show=False)

## Value Histograms (for each dimension)

In [95]:
def generate_value_histograms(ballots, weights, centroids, dims, bins, embedding_type, save_location, show=True):
    n_clusters = centroids.shape[0]
    expanded_ballots = expand_weighted_ballots(ballots, weights)
    distances = cdist(expanded_ballots, centroids, metric='cityblock')
    nearest_centroid = np.argmin(distances, axis=1)
    
    fig, axes = plt.subplots(len(dims), n_clusters + 1, figsize=(12*n_clusters, 4*len(dims)))
    if len(dims) == 1:
        axes = [axes]
    if n_clusters == 1:
        axes = [axes]

    if embedding_type == 'hh':
        embedding_name = 'Head-to-Head'
    elif embedding_type == 'bordaA':
        embedding_name = 'Borda Average'
    else:
        embedding_name = 'Borda Pessimistic'


    dim_to_pair = f = [(i,j) for i in range(1, num_cands+1) for j in range(i+1, num_cands + 1)]
    for i, dim in enumerate(dims):
        values = expanded_ballots[:, dim]
        ax = axes[i][0] if len(dims) > 1 else axes[0]
        histplot(values, ax=ax, bins=bins, stat='proportion')
        candidate_title = f'Candidate {dim_to_pair[dim][0]} (1) vs. Candidate {dim_to_pair[dim][1]} (-1)' if embedding_type == 'hh' else f'Candidate {dim+1}'
        ax.set_title(f'All Ballots Score Distribution for {candidate_title} ({embedding_name})')
        # Fix x-ticks for HH
        if embedding_type == 'hh':
            ax.set_xticks([-1, 0, 1])
        ax.set_xlabel('Score')
        ax.set_ylabel('Proportion')
        for j in range(n_clusters):
            ax = axes[i][j+1] if len(dims) > 1 else axes[j]
            cluster_values = values[nearest_centroid == j]
            histplot(cluster_values, ax=ax, bins=bins, stat='density')
            ax.set_title(f'Cluster {j+1} Score Distribution for {candidate_title} ({embedding_name})')
            # Fix x-ticks for HH
            if embedding_type == 'hh':
                ax.set_xticks([-1, 0, 1])
            ax.set_xlabel('Score')
            ax.set_ylabel('Proportion')
    
    plt.tight_layout()
    plt.savefig(f'{save_location}.png')
    if show:
        plt.show()
    plt.close()

In [97]:
VALUE_HISTOGRAMS_DIRECTORY = f'{RESULTS_DIRECTORY}/value_histograms'
bin_edges = {
    'hh': np.arange(-1.5, 2, 1),
    'bordaP': np.arange(-0.5, num_cands, 1),
    'bordaA': np.arange(-0.25, num_cands-0.5, 0.5)
}
print(bin_edges['hh'])
print(bin_edges['bordaP'])
print(bin_edges['bordaA'])
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            generate_value_histograms(BALLOTS[embedding_type], weights, centroids[(model_type, embedding_type, num_clusters)], DIMS[embedding_type], bin_edges[embedding_type], embedding_type, f"{VALUE_HISTOGRAMS_DIRECTORY}/{model_name}_value_hists", show=False)

[-1.5 -0.5  0.5  1.5]
[-0.5  0.5  1.5  2.5  3.5  4.5  5.5  6.5]
[-0.25  0.25  0.75  1.25  1.75  2.25  2.75  3.25  3.75  4.25  4.75  5.25
  5.75  6.25]


In [99]:
def plot_distance_scatter(ballots, weights, centroids, save_location, show=True):
    # Assume 2 clusters (2 dimensions for scatter plot)
    distances = cdist(ballots, centroids, metric='cityblock')

    # Plot scatter plot where x axis is distance to centroid 1 and y axis is distance to centroid 2
    plt.figure(figsize=(10, 10))
    plt.scatter(distances[:, 0], distances[:, 1], s=weights, alpha=0.5)
    # Plot x=y across the whole scatter plot
    plt.plot([0, np.max(distances)], [0, np.max(distances)], color='red')
    plt.xlabel('Distance to Centroid 1')
    plt.ylabel('Distance to Centroid 2')
    plt.title('Distance to Centroids Scatter Plot')
    plt.grid(True, alpha=0.3)
    plt.savefig(f'{save_location}.png')
    if show:
        plt.show()
    plt.close()

In [100]:
DISTANCE_SCATTER_DIRECTORY = f'{RESULTS_DIRECTORY}/distance_scatter'
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            plot_distance_scatter(BALLOTS[embedding_type], weights, centroids[(model_type, embedding_type, num_clusters)], f"{DISTANCE_SCATTER_DIRECTORY}/{model_name}_dist_scatter", show=False)

## MDS Plots

In [101]:
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from seaborn import scatterplot

def plot_2d_mds(ballots, centroids, weights, show=True, save_name=None, threshold=10, max_iter=300, random_state=42):
    mds = MDS(n_components=2, metric=True, dissimilarity='precomputed', max_iter=max_iter, random_state=random_state)
    combined = np.vstack([ballots, centroids])
    n_samples = ballots.shape[0] + centroids.shape[0]

    pruned_ballots = ballots[weights >= threshold]
    combined = np.vstack([pruned_ballots, centroids])
    pruned_weights = weights[weights >= threshold]

    dissimilarity = manhattan_distances(combined)

    X = mds.fit_transform(dissimilarity)
    X_points = X[:len(pruned_ballots)]
    X_centroids = X[len(pruned_ballots):]
    
    assignments = assign_to_clusters(X_points, X_centroids)
    plt.figure(figsize=(10, 8))
    

    plt.scatter(X_points[:, 0], X_points[:, 1], 
               c=assignments, s=pruned_weights, cmap='bwr', alpha=0.6)
    

    plt.scatter(X_centroids[:, 0], X_centroids[:, 1], 
               c='gold', s=200, marker='*', label='Centroids', alpha=0.6)
    
    plt.title('MDS')
    plt.xlabel('First MDS dimension')
    plt.ylabel('Second MDS dimension')
    plt.legend()
    if save_name:
        plt.savefig(save_name)
    if show:
        plt.show()
    plt.close()

In [102]:
MDS_DIRECTORY = f'{RESULTS_DIRECTORY}/mds'
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            plot_2d_mds(BALLOTS[embedding_type], centroids[(model_type, embedding_type, 2)], weights, show=False, save_name=f"{MDS_DIRECTORY}/{model_name}_mds.png")



## Fraction Equidistant and Silhouette Scores

In [103]:
def fraction_equidistant(ballots, weights, centroids, tolerance=0):
    """
    Calculate the fraction of equidistant points from each centroid.
    
    Parameters:
    ballots : numpy.ndarray
        A matrix where each row represents an individual ballot
        Shape: (total_ballots, n_candidates)
    weights : numpy.ndarray
        An array containing the count/weight of each ballot pattern
        Shape: (total_ballots,)
    centroids : numpy.ndarray
        Array of shape (n_clusters, n_dimensions) containing centroid coordinates
    tolerance : float, optional
        Numerical tolerance for considering distances equal (default: 1e-10)
        
    Returns:
    --------
    numpy.ndarray
        Array of shape (n_clusters,) containing the fraction of equidistant points
        for each centroid
    """
    # Calculate distances between each ballot and each centroid
    # Returns matrix of shape (total_ballots, n_clusters)
    distances = cdist(ballots, centroids, metric='cityblock')
    
    # For each ballot, find if it's equidistant to any pair of centroids
    n_clusters = len(centroids)
    equidistant_counts = np.zeros(n_clusters)
    total_weight = np.sum(weights)
    
    # For each ballot pattern
    for i, weight in enumerate(weights):
        # Get distances from this ballot to all centroids
        point_distances = distances[i]
        
        # For each centroid
        for j in range(n_clusters):
            # Check if this point is equidistant between this centroid and any other
            for k in range(j + 1, n_clusters):
                # If distances are equal (within tolerance)
                if abs(point_distances[j] - point_distances[k]) <= tolerance:
                    # Add the weight to both centroids' counts
                    equidistant_counts[j] += weight
                    equidistant_counts[k] += weight
                    
    # Convert counts to fractions by dividing by total weight
    fractions = equidistant_counts / (2 * total_weight)
    
    return fractions

In [11]:
for model_type in model_types:
    for embedding_type in embedding_types:
        for num_clusters in range(2, 3):
            model_name = f'{model_type}_{embedding_type}_{num_clusters}'
            print(model_name)
            print("Fraction Equidistant")
            print(fraction_equidistant(BALLOTS[embedding_type], weights, centroids[model_type, embedding_type, num_clusters], tolerance=0))
            expanded_ballots = expand_weighted_ballots(BALLOTS[embedding_type], weights)
            labels = assign_to_clusters(expanded_ballots, centroids[model_type, embedding_type, num_clusters])
            print("Silhouette Score")
            print(silhouette_score(expanded_ballots, labels))
            print()


continuous_hh_2
Fraction Equidistant
[0. 0.]
Silhouette Score
0.3678435419393745

continuous_bordaP_2
Fraction Equidistant
[0.00349094 0.00349094]
Silhouette Score
0.4022093213948889

continuous_bordaA_2
Fraction Equidistant
[0. 0.]
Silhouette Score
0.41950686018033584

continuous_rest_hh_2
Fraction Equidistant
[0.01162174 0.01162174]
Silhouette Score
0.3737182176012703

continuous_rest_bordaP_2
Fraction Equidistant
[0. 0.]
Silhouette Score
0.4024611329696099

continuous_rest_bordaA_2
Fraction Equidistant
[0.08281043 0.08281043]
Silhouette Score
0.41656121686560393

discrete_hh_2
Fraction Equidistant
[0.0048608 0.0048608]
Silhouette Score
0.36641216675437344

discrete_bordaP_2
Fraction Equidistant
[0. 0.]
Silhouette Score
0.4024611329696099

discrete_bordaA_2
Fraction Equidistant
[0.01113566 0.01113566]
Silhouette Score
0.419429132155026



In [107]:
def head_to_head_to_borda(proxy_vector, num_candidates):
    """
    Convert a head-to-head proxy vector to a Borda count vector (pessimistic style).
    
    Args:
        proxy_vector: List of head-to-head comparisons in order (1,2), (1,3), ..., (1,n), (2,3), ...
        num_candidates: Total number of candidates
        
    Returns:
        List of Borda scores for each candidate (0-based indexing)
    """
    # Initialize Borda scores
    borda_scores = [0] * num_candidates
    
    # Process each head-to-head comparison
    idx = 0
    for i in range(num_candidates):
        for j in range(i + 1, num_candidates):
            comparison = proxy_vector[idx]
            
            # Update Borda scores based on head-to-head result
            if comparison == 1:  # i wins against j
                borda_scores[i] += 1
            elif comparison == -1:  # j wins against i
                borda_scores[j] += 1
            # If comparison == 0, it's a tie, no points awarded
            
            idx += 1
            
    
    return borda_scores

In [111]:
hh_cont_rest = centroids['continuous_rest', 'hh', 2]
head_to_head_to_borda(hh_cont_rest[1], num_cands)
print(hh_cont_rest[0])

[ 1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1.  1. -1. -1.  1. -1.  1.
 -1. -1.  1.]


In [112]:
CENTROIDS_SILHOUETTE_FRACTION_EQ_DIRECTORY = f"{RESULTS_DIRECTORY}/centroids_silhouette_fraction_eq.txt"
with open(CENTROIDS_SILHOUETTE_FRACTION_EQ_DIRECTORY, 'w') as f:
    for model_type in model_types:
        for embedding_type in embedding_types:
            for num_clusters in range(2, 3):
                model_name = f'{model_type}_{embedding_type}_{num_clusters}'
                f.write(f"{model_name}\n")
                f.write("Centroids\n")
                f.write(f"{centroids[model_type, embedding_type, num_clusters]}\n")
                if embedding_type == 'hh':
                    f.write(f"Converted to Borda Pessimistic\n")
                    f.write(f"{head_to_head_to_borda(centroids[model_type, embedding_type, num_clusters][0], num_cands)}\n")
                    f.write(f"{head_to_head_to_borda(centroids[model_type, embedding_type, num_clusters][1], num_cands)}\n")
                f.write("Fraction Equidistant\n")
                fraction_eq = fraction_equidistant(BALLOTS[embedding_type], weights, centroids[model_type, embedding_type, num_clusters], tolerance=0)
                f.write(f"{fraction_eq}\n")
                expanded_ballots = expand_weighted_ballots(BALLOTS[embedding_type], weights)
                labels = assign_to_clusters(expanded_ballots, centroids[model_type, embedding_type, num_clusters])
                f.write("Silhouette Score\n")
                silhouette = silhouette_score(expanded_ballots, labels)
                f.write(f"{silhouette}\n\n")
